In [111]:
import pandas as pd
import plotly.express as px

In [112]:
# -- Import data from CSV files --

df_order_items = pd.read_csv('..\data\postgresql_order_items_exported.csv')
df_orders = pd.read_csv('..\data\postgresql_orders_exported.csv')

<>:3: SyntaxWarning:

invalid escape sequence '\d'

<>:4: SyntaxWarning:

invalid escape sequence '\d'

<>:3: SyntaxWarning:

invalid escape sequence '\d'

<>:4: SyntaxWarning:

invalid escape sequence '\d'

C:\Users\Rirhc\AppData\Local\Temp\ipykernel_24616\3079852966.py:3: SyntaxWarning:

invalid escape sequence '\d'

C:\Users\Rirhc\AppData\Local\Temp\ipykernel_24616\3079852966.py:4: SyntaxWarning:

invalid escape sequence '\d'



In [113]:
# -- Initialize value --

platform_color = {
    "Tiktok" : "#FE2C55",
    "Shopee" : "#EE4D2D",
    "Lazada" : "#0f146d"
}

category = {
    "001" : "Tire",
    "002": "Rubber",
    "003": "Motor Oil",
    "004": "Battery",
    "005": "Spark Plug",
    "006": "Sprocket",
    "007": "Motorcycle Belt",
    "008": "Etc",
    "009": "Brake"
}

In [114]:
# -- Filter columns --
df_order_items = df_order_items[['order_id', 'selling_sku_id', 'quantity', 'unit_price', 'subtotal']]
df_orders = df_orders[['order_id', 'platform_name', 'order_date', 'total_amount']]

# Add category_id column
df_order_items['category_id'] = df_order_items['selling_sku_id'].str[:3]

df_orders['order_date'] = pd.to_datetime(df_orders['order_date'])
df_orders['order_month'] = df_orders['order_date'].dt.to_period('M').astype(str)

In [115]:
total_order = df_orders['order_id'].nunique()
total_revenue = df_orders['total_amount'].sum()
average_order_value = total_revenue / total_order
basket_size = df_order_items['quantity'].sum() / df_orders['order_id'].nunique()
print(total_order, total_revenue, average_order_value, basket_size)

1849 2103885.0 1137.850189291509 1.4618712817739319


In [116]:
platform_count = df_orders['platform_name'].value_counts(normalize=True) 
platform_ratio = platform_count * total_order

fig = px.pie(
    platform_ratio,
    names=platform_ratio.index,
    values=platform_ratio.values,
    title='Orders by Platform',
    color=platform_ratio.index,
    color_discrete_map=platform_color
)

fig.update_traces(textinfo='value+percent')
fig.show()

In [117]:
platform_sales = df_orders.groupby('platform_name')['total_amount'].sum()

fig = px.pie(
    platform_sales,
    names=platform_sales.index,
    values=platform_sales.values,
    title='Orders by Platform',
    color=platform_sales.index,
    color_discrete_map=platform_color
)

fig.update_traces(textinfo='value+percent')
fig.show()

In [118]:
sales_by_month = df_orders.groupby(['order_month', 'platform_name'])['total_amount'].sum().reset_index()


fig = px.line(
    sales_by_month,
    x='order_month',
    y='total_amount',
    color='platform_name',
    title='Monthly Sales by Platform',
    color_discrete_map=platform_color,
    markers=True
)

fig.show()

In [119]:
orders_by_month = df_orders.groupby(['order_month', 'platform_name'])['order_id'].count().reset_index()
orders_by_month.rename(columns={'order_id': 'number_of_orders'}, inplace=True)

fig = px.line(
    orders_by_month,
    x='order_month',
    y='number_of_orders',
    color='platform_name',
    title='Monthly Orders by Platform',
    color_discrete_map=platform_color,
    markers=True
)

fig.show()

In [129]:
aov = sales_by_month.merge(
    orders_by_month, 
    on=['order_month', 'platform_name'])

aov['average_order_value'] = aov['total_amount'] / aov['number_of_orders']

fig = px.line(
    aov,
    x='order_month',
    y='average_order_value',
    color='platform_name',
    title='Average Order Value by Platform',
    color_discrete_map=platform_color,
    markers=True
)

fig.show()

In [120]:
platform_aov = platform_sales / platform_ratio
platform_aov

platform_name
Lazada     805.118056
Tiktok    1350.044287
dtype: float64

In [121]:
platform = ["Lazada", "Tiktok"]

# Platform Filtered
platform_filtered_orders =  df_orders[df_orders['platform_name'].isin(platform)]
platform_filtered_order_items = df_order_items[df_order_items['order_id'].isin(platform_filtered_orders['order_id'])]

# Category Sales
category_sales = platform_filtered_order_items.groupby('category_id')['subtotal'].sum().sort_values(ascending=False)
category_sales

category_id
001    1804870.0
003     230104.0
004      45338.0
002      17261.0
008       6312.0
Name: subtotal, dtype: float64

In [122]:
Most_Saled_Product_unit = platform_filtered_order_items.groupby('selling_sku_id')['quantity'].sum().sort_values(ascending=False)
Most_Saled_Product_unit

selling_sku_id
003-VALVOLINE-VR1-10W50-1000-001    303
003-VALVOLINE-VR1-10W40-1000-001    227
001-IRC-IZSS-TL-1307013-PCX160      107
001-IRC-MAXING-TT-25017-SET          93
001-IRC-MAXING-TT-22517-SET          91
                                   ... 
002-IRC-0-0-1309015-PHANTOM           1
001-QUICK-GR750-TT-27517-SET          1
001-QUICK-GR750-TT-25017-SET          1
003-VALVOLINE-CHAMP-4T-1000-012       1
008-HONDA-ENGCLN-0-800-001            1
Name: quantity, Length: 133, dtype: int64

In [123]:
Most_Saled_Product_value = platform_filtered_order_items.groupby('selling_sku_id')['subtotal'].sum().sort_values(ascending=False)   
Most_Saled_Product_value

selling_sku_id
001-IRC-IZSS-TL-1307013-PCX160    192960.0
001-IRC-IZSS-TL-1407014-XMAX      153533.0
001-IRC-IZSS-TL-1107014-PCX160    100870.0
001-IRC-IZSS-TL-1207015-XMAX       92519.0
001-IRC-WINGS-TL-1207014-001       83874.0
                                    ...   
001-QUICK-GR730-TT-709017-001        280.0
001-QUICK-GR730-TT-609017-001        259.0
002-IRC-0-0-40017-001                235.0
008-HONDA-ENGCLN-0-800-001           119.0
002-IRC-0-0-27514-001                117.0
Name: subtotal, Length: 133, dtype: float64

In [124]:
fig = px.line(
    
)